In [288]:
from konlpy.tag import Twitter
from gensim.models import Word2Vec
import csv
from konlpy.tag import Twitter
from konlpy.tag import Hannanum
from konlpy.tag import Kkma
from konlpy.tag import Komoran
from konlpy.tag import Okt
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
import tensorflow as tf
import matplotlib

import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

import csv
import logging
from konlpy import jvm
logger = logging.getLogger(__name__)
jvm.init_jvm()



In [289]:
data_set = pd.read_excel("data.xlsx")
data_set = data_set.iloc[:,1:]
data_set.rename({0:'news'},axis=1, inplace= True)
data_label = {1:'culture',2:'global',3:'politic',4:'society',5:'economy'}




In [290]:
for i in range(len(data_set)) :
    data_set['news'][i] = re.sub('[^a-zA-Z0-9 ㄱ-ㅣ가-힣]', '', data_set['news'][i])
    


C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [291]:
data_set = data_set.sample(frac=1).reset_index(drop=True)

In [292]:
data_set = data_set.iloc[:40000,:]  # 5만 이상 데이터는 오류 발생(왜?)

In [293]:
data_set.head()

,news,label
0,추하다 vs 갈라서자바른미래 갈등 폭발,3
1,서울경기 호우주의보호우경보로 격상 시간당 50mm 이상 폭우,1
2,갤럭시 폴드 불량강화유리 떼내고 고장났다 떼쓰는 격,5
3,신종 코로나 불안감에 쿠팡 주문량 역대 최대김범석 비상 상황,1
4,삼성 퇴직 임원도 당했다DLS 손실 눈덩이,5


In [294]:
data_set.isna().sum()

news     0
label    0
dtype: int64

In [295]:
def get_noun(text):
    tokenizer = Twitter()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun2(text):
    tokenizer = Hannanum()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_noun3(text):
    tokenizer = Okt()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]

def get_morphs3(text):
    tokenizer = Okt()
    morphs = tokenizer.morphs(text)
    return [n for n in morphs]

def get_noun4(text):
    tokenizer = Komoran()
    nouns = tokenizer.nouns(text)
    return [n for n in nouns]


In [296]:
train_set, test_set = train_test_split(data_set, test_size = 0.3, random_state = 42)

In [297]:
train_set=train_set.reset_index(drop=True)
test_set=test_set.reset_index(drop=True)

In [298]:
#fit_transform = CV.fit_transform(train_set["news"])

In [299]:
#label_set = data_set['label']
label_set = train_set['label']
len(label_set)

28000

In [300]:
train_set

,news,label
0,날씨 구름 많고 포근한 하루미세먼지 농도 짙어요,1
1,해외 이모저모 내 아기 놔줘 만류에도 위험한 침례 강행,2
2,모든 자영업자 산재보험 가입당정청 사각지대 해소방안,3
3,와이파이 여론 역풍에한걸음씩 물러선 현대차 노사,5
4,정국 혼란 베네수엘라에 정전사태학교 휴업병원 발동동,2
...,...,...
27995,유시민 살인자 북송 비난 자기 집 방 하나 내주든가,3
27996,친구 흉기로 살해 초등생 여아 형사처벌 안 받나 괴롭힘 당해,4
27997,여야4당 대표 봉하마을 집결노무현 10주기 추도식 참석,3
27998,저축은행 사태 트라우마 증권사 부동산 PF 쏠림에 제동 건 금,5


In [301]:
from konlpy.tag import Twitter
from konlpy.tag import Kkma
tf.reset_default_graph()
twitter = Twitter()
token = []
embeddingmodel = []
for i in range(len(data_set)):
    label = data_set['label']
    sentence = twitter.pos(data_set['news'][i], norm=True, stem=True)
    temp = []
    temp_embedding = []
    all_temp = []
    for k in range(len(sentence)):
        temp_embedding.append(sentence[k][0])
        temp.append(sentence[k][0] + '/' + sentence[k][1])
    all_temp.append(temp)
    embeddingmodel.append(temp_embedding)
    
    all_temp.append(label[i])
    token.append(all_temp)
    
embeddingmodel = []
for i in range(len(label_set)):
    temp_embeddingmodel = []
    for k in range(len(token[i][0])):
        temp_embeddingmodel.append(token[i][0][k])
    embeddingmodel.append(temp_embeddingmodel)
embedding  = Word2Vec(embeddingmodel, size = 300, window =5, min_count = 10,
                      iter = 5, sg=1)

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [302]:
embedding.most_similar('코로나/Noun')

C:\Users\ICT01_18\Anaconda3\envs\tf_test\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('신종/Noun', 0.9829598665237427),
 ('우한/Noun', 0.9699337482452393),
 ('폐렴/Noun', 0.9631495475769043),
 ('확/Noun', 0.9496662616729736),
 ('코로나바이러스/Noun', 0.9424995183944702),
 ('국내/Noun', 0.9265066385269165),
 ('전세기/Noun', 0.9231536388397217),
 ('번째/Suffix', 0.9182091951370239),
 ('진자/Noun', 0.9165610671043396),
 ('확진/Noun', 0.9163594245910645)]

In [303]:
print(token[0][0])
print(token[1][0])
print(token[2][0])
data_set


['추하다/Adjective', 'vs/Alpha', '갈라/Noun', '서자/Noun', '바른/Modifier', '미래/Noun', '갈등/Noun', '폭발/Noun']
['서울/Noun', '경기/Noun', '호우/Noun', '주의보/Noun', '호우경보/Noun', '로/Josa', '격상/Noun', '시간/Noun', '당/Suffix', '50/Number', 'mm/Alpha', '이상/Noun', '폭우/Noun']
['갤럭시/Noun', '폴드/Noun', '불량/Noun', '강화유리/Noun', '떼다/Verb', '고장/Noun', '나다/Verb', '떼쓰다/Verb', '격/Noun']


,news,label
0,추하다 vs 갈라서자바른미래 갈등 폭발,3
1,서울경기 호우주의보호우경보로 격상 시간당 50mm 이상 폭우,1
2,갤럭시 폴드 불량강화유리 떼내고 고장났다 떼쓰는 격,5
3,신종 코로나 불안감에 쿠팡 주문량 역대 최대김범석 비상 상황,1
4,삼성 퇴직 임원도 당했다DLS 손실 눈덩이,5
...,...,...
39995,오늘부터 장마 시작중부는 낮까지 폭염,4
39996,환자 밀접접촉자도 아닌데국내 첫 2차 감염,4
39997,음원 사재기 의혹 닐로 SNS 댓글 차단,4
39998,풀영상 이인영 한국당 비례대표 폐지 주장은 어깃장,3


In [304]:
def One_hot(data):
       
    index_dict = {value:index for index,value in enumerate(set(data))}
    result = []

    for value in data:

        one_hot = np.zeros(len(index_dict))
        index = index_dict[value]
        one_hot[index] = 1
        result.append(one_hot)

    return result


def Convert2Vec(model_name, doc): 
    #train_X_ = W2V.Convert2Vec("Word2Vec_csv_article.embedding",train_X)
    word_vec = []
    model = model_name
    for sent in doc:
        sub = []
        for word in sent:
            if word in model.wv.vocab:
                sub.append(model.wv[word]) 
            else:
                sub.append(np.random.uniform(-0.25,0.25,300)) # 사전에 해당 워드가 없으면 랜덤한 vector 생성하게 해줌
        word_vec.append(sub)

    return word_vec





In [305]:
class Bi_LSTM():
    
    def __init__(self, lstm_units, num_class, keep_prob):
        
        self.lstm_units = lstm_units
        
        with tf.variable_scope('forward', reuse = tf.AUTO_REUSE):
            
            self.lstm_fw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_fw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
            
        with tf.variable_scope('backward', reuse = tf.AUTO_REUSE):
            
            self.lstm_bw_cell = tf.nn.rnn_cell.LSTMCell(lstm_units, forget_bias=1.0, state_is_tuple=True)
            self.lstm_bw_cell = tf.contrib.rnn.DropoutWrapper(self.lstm_fw_cell, output_keep_prob = keep_prob)
        
        with tf.variable_scope('Weights', reuse = tf.AUTO_REUSE):
           
            self.W = tf.get_variable(name="W", shape=[2 * lstm_units, num_class],
                                dtype=tf.float32, initializer = tf.contrib.layers.xavier_initializer())
            self.b = tf.get_variable(name="b", shape=[num_class], dtype=tf.float32,
                                initializer=tf.zeros_initializer())
            
            
    def logits(self, X, W, b, seq_len):
        
        (output_fw, output_bw), states = tf.nn.bidirectional_dynamic_rnn(self.lstm_fw_cell, self.lstm_bw_cell,dtype=tf.float32,
                                                                            inputs = X, sequence_length = seq_len)

        outputs = tf.concat([states[0][1], states[1][1]], axis=1)
        pred = tf.matmul(outputs, W) + b        
        return pred
        
    def model_build(self, logits, labels, learning_rate = 0.001):
        
        with tf.variable_scope("loss"):
            
            loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = logits , labels = labels)) # Softmax loss
            optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss) 
            merged = tf.summary.merge_all()
        return loss, optimizer, merged
    
    def graph_build(self, avg_loss, avg_acc):
        
        tf.summary.scalar('Loss', avg_loss)
        tf.summary.scalar('Accuracy', avg_acc)
        merged = tf.summary.merge_all()
        return merged

In [306]:
def Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size):
        
    zero_pad = np.zeros((Batch_size, Maxseq_length, Vector_size))
    for i in range(Batch_size):
        zero_pad[i, :np.shape(train_batch_X[i])[0], :np.shape(train_batch_X[i])[1] ] = train_batch_X[i]
    return zero_pad

In [307]:
tokens = np.array(token)
data_x = tokens[:,0]
data_y = tokens[:,1]


data_y = One_hot(data_y)
data_x = Convert2Vec(embedding, data_x)

train_x, test_x, train_y, test_y = train_test_split(data_x,data_y,
                                                    test_size = 0.3,
                                                    random_state = 42)
print(len(train_x))
print(len(train_y))
print(len(test_x))
#train_x = train_x.reset_index(drop=True)
#train_y = train_y.reset_index(drop=True)
#test_x = test_x.reset_index(drop=True)
#test_y = test_y.reset_index(drop=True)

28000
28000
12000


In [308]:
len(data_x[0][0])

300

In [309]:
tf.reset_default_graph()
Batch_size = 32
Total_size = len(train_x)
Vector_size = 300
seq_length = [len(x) for x in train_x]
Maxseq_length = max(seq_length)
learning_rate = 0.001
lstm_units = 128
num_class = 5
training_epochs = 5
keep_prob = 0.75

X = tf.placeholder(tf.float32, shape = [None, Maxseq_length, Vector_size])
Y = tf.placeholder(tf.float32, shape = [None, num_class])
seq_len = tf.placeholder(tf.int32 , shape = [None])

BiLSTM = Bi_LSTM(lstm_units, num_class, keep_prob)
with tf.variable_scope('loss', reuse = tf.AUTO_REUSE) :
    logits = BiLSTM.logits(X, BiLSTM.W, BiLSTM.b, seq_len)
    loss, optimizer, merged = BiLSTM.model_build(logits, Y, learning_rate)

prediction = tf.nn.softmax(logits)
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

init = tf.global_variables_initializer()

total_batch = int(Total_size / Batch_size)

print("Start training!")

#modelName = "BiLSTM.model"
saver = tf.train.Saver()

Start training!


In [310]:
test_set

,news,label
0,부모 학대 못 이긴 우크라이나 8세 소년 극단적 선택,2
1,NHK 미사일 오보에전문가들 오보로 전쟁 날 수도,2
2,바른미래 퇴진파 권은희 최고위원직 박탈이유는 당비 미납,3
3,가족사진같은 국무위 단체사진김영철 중앙배치 눈길,3
4,진주 한 아파트서 방화살인사건주민 5명 숨지고 5명 중경상 등,4
...,...,...
11995,르포 고양 창릉 서울서 15분 욕망의 논밭 들썩,5
11996,카페 점령한 카공족 더 속상한 건 이겁니다,1
11997,제네시스 GV80 30 디젤 출시6580만원부터,1
11998,박용만 일본도 보고 있는데최선 다해 대통령 도울 때,5


In [311]:
print(test_set['news'])
tokens[2][0]

0              부모 학대 못 이긴 우크라이나 8세 소년 극단적 선택
1                NHK 미사일 오보에전문가들 오보로 전쟁 날 수도
2             바른미래 퇴진파 권은희 최고위원직 박탈이유는 당비 미납
3                가족사진같은  국무위 단체사진김영철 중앙배치 눈길
4        진주 한 아파트서 방화살인사건주민 5명 숨지고 5명 중경상 등 
                        ...                 
11995             르포 고양 창릉 서울서 15분 욕망의 논밭 들썩
11996                카페 점령한 카공족 더 속상한 건 이겁니다
11997             제네시스 GV80 30 디젤 출시6580만원부터
11998           박용만 일본도 보고 있는데최선 다해 대통령 도울 때
11999          동생은 이승 형은 저승 관장그럼 저승이 더 좋다는 뜻
Name: news, Length: 12000, dtype: object


['갤럭시/Noun',
 '폴드/Noun',
 '불량/Noun',
 '강화유리/Noun',
 '떼다/Verb',
 '고장/Noun',
 '나다/Verb',
 '떼쓰다/Verb',
 '격/Noun']

In [ ]:
import time
import os
result  = []
loss_graph = []
loss_graph_test=[]
with tf.Session() as sess:
    #writer = tf.summary.FileWriter('', sess.graph)
    start_time = time.time()
    sess.run(init)
    train_writer = tf.summary.FileWriter('Bidirectional_LSTM', sess.graph)
    i = 0
    for epoch in range(training_epochs):
        #try :
            avg_acc, avg_loss = 0. , 0.
            for step in range(total_batch):

                train_batch_X = train_x[step*Batch_size : step*Batch_size+Batch_size]

                train_batch_Y = train_y[step*Batch_size : step*Batch_size+Batch_size]
                batch_seq_length = seq_length[step*Batch_size : step*Batch_size+Batch_size]
                train_batch_X = Zero_padding(train_batch_X, Batch_size, Maxseq_length, Vector_size)
                
                #summary, _ = sess.run([merged,optimizer], feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})

                sess.run(optimizer, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                
                loss_ = sess.run(loss, feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_loss += loss_ / total_batch

                acc = sess.run(accuracy , feed_dict={X: train_batch_X, Y: train_batch_Y, seq_len: batch_seq_length})
                avg_acc += acc / total_batch
                #train_writer.add_summary(summary, step)
            print("epoch : {:02d} step : {:04d} loss = {:.6f} accuracy= {:.6f}".format(epoch+1, step+1, loss_, acc))
            #print("Test Accuracy : ", sess.run(accuracy, feed_dict={X:test_x, Y:test_y}))
            #summary = sess.run(BiLSTM.graph_build(avg_loss, avg_acc))       
            #train_writer.add_summary(summary, i)
            #merged = BiLSTM.graph_build()
            i += 1
        #except : 
            #print('Error Epoch : ', epoch)
            loss_graph.append([acc,epoch])
            #acc = sess.run(accuracy , feed_dict={X: test_x, Y: test_y, seq_len: batch_seq_length})
            #print('Test Accuracy : ', acc)

    duration = time.time() - start_time
    minute = int(duration / 60)
    second = int(duration) % 60
    print("%dminutes %dseconds" % (minute,second))
    save_path = saver.save(sess, os.getcwd())

    train_writer.close()
    print('save_path',save_path)
    
    test_size = len(test_x)
    test_batch = int(test_size / Batch_size)
    seq_length = [len(x) for x in test_x]
    keep_prob = 1.0
    total_acc = 0
    for step in range(test_batch):
        test_batch_X = test_x[step*Batch_size: step*Batch_size + Batch_size]
        test_batch_Y = test_y[step*Batch_size: step*Batch_size + Batch_size]
        batch_seq_length = seq_length[step*Batch_size : step*Batch_size + Batch_size]
        test_batch_X = Zero_padding(test_batch_X, Batch_size, Maxseq_length, Vector_size)
        
        #summary, _ = sess.run([mreged, optimizer], feed_dict = {X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        acc = sess.run(accuracy , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
        total_acc += acc/test_batch
        pred_v = sess.run(prediction , feed_dict={X: test_batch_X, Y: test_batch_Y, seq_len: batch_seq_length})
 
        print("loss = {:.6f} accuracy = {:.6f}".format(loss_, acc), "/ step =",step)
            
            
    print(" Total Accuracy : {}".format(total_acc))
    result.append(total_acc)

    
print("----------------------------------------")
print("BI-LSTM finish")
for i in range(0, len(result)):
    print("epoch : ", i+1, " / accuracy : ", result[i])
    
print("\nBest Result epoch : ", result.index(max(result)) + 1, " / accuracy: ", max(result))

epoch : 01 step : 0875 loss = 0.975620 accuracy= 0.625000
